In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from datetime import datetime
import os
import sys


In [3]:
base_dir = os.path.abspath('../')
file_path = os.path.join(base_dir, 'data', 'processed', 'cleaned_data.csv')
df = pd.read_csv(file_path)

In [4]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


In [6]:
# Convert TransactionStartTime to datetime format
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   TransactionId         95662 non-null  object             
 1   BatchId               95662 non-null  object             
 2   AccountId             95662 non-null  object             
 3   SubscriptionId        95662 non-null  object             
 4   CustomerId            95662 non-null  object             
 5   CurrencyCode          95662 non-null  object             
 6   CountryCode           95662 non-null  int64              
 7   ProviderId            95662 non-null  object             
 8   ProductId             95662 non-null  object             
 9   ProductCategory       95662 non-null  object             
 10  ChannelId             95662 non-null  object             
 11  Amount                95662 non-null  float64            
 12  Valu

In [12]:
df.columns.to_list()

['TransactionId',
 'BatchId',
 'AccountId',
 'SubscriptionId',
 'CustomerId',
 'CurrencyCode',
 'CountryCode',
 'ProviderId',
 'ProductId',
 'ProductCategory',
 'ChannelId',
 'Amount',
 'Value',
 'TransactionStartTime',
 'PricingStrategy',
 'FraudResult']

In [14]:
# Step 1: Create Aggregate Features
agg_features = df.groupby('CustomerId').agg({
    'Value': ['sum', 'mean', 'count', 'std']
}).reset_index()

# Rename columns for clarity
agg_features.columns = ['CustomerId', 'Total_Transaction_Amount', 'Avg_Transaction_Amount', 'Transaction_Count', 'Std_Transaction_Amount']

In [15]:
# Step 2: Extract Date-Time Features
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
df['Transaction_Hour'] = df['TransactionStartTime'].dt.hour
df['Transaction_Day'] = df['TransactionStartTime'].dt.day
df['Transaction_Month'] = df['TransactionStartTime'].dt.month
df['Transaction_Year'] = df['TransactionStartTime'].dt.year

In [17]:
# Step 3: Encode Categorical Variables
categorical_columns = ['ProviderId', 'ProductCategory', 'ChannelId', 'PricingStrategy']

# Label Encoding for high-cardinality features
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoders for reference

In [18]:
# Step 4: Normalize/Standardize Numerical Features
scaler = StandardScaler()  # Use StandardScaler for better model performance
numeric_features = ['Total_Transaction_Amount', 'Avg_Transaction_Amount', 'Transaction_Count', 'Std_Transaction_Amount']
agg_features[numeric_features] = scaler.fit_transform(agg_features[numeric_features])

In [20]:

# Step 6: Construct Default Estimator (RFMS-based proxy)
# Recency: Days since last transaction per customer
recency = df.groupby('CustomerId')['TransactionStartTime'].max().reset_index()
recency['Recency'] = (df['TransactionStartTime'].max() - recency['TransactionStartTime']).dt.days

# Frequency: Number of transactions per customer (already calculated in Transaction_Count)
frequency = df.groupby('CustomerId')['TransactionId'].count().reset_index()
frequency.columns = ['CustomerId', 'Frequency']

# Monetary: Total money spent per customer (already calculated in Total_Transaction_Amount)
monetary = agg_features[['CustomerId', 'Total_Transaction_Amount']]

# Merge RFMS Features
rfms_df = recency.merge(frequency, on='CustomerId').merge(monetary, on='CustomerId')

# Define High and Low RFMS Score Groups (Simple Thresholding)
rfms_df['RFMS_Score'] = (rfms_df['Frequency'] > rfms_df['Frequency'].median()).astype(int)

In [21]:
# Step 7: Perform Weight of Evidence (WoE) Binning
def calculate_woe_iv(df, feature, target):
    bins = pd.qcut(df[feature], q=10, duplicates='drop')  # Divide into 10 bins
    grouped = df.groupby(bins)[target].agg(['count', 'sum'])
    grouped['good'] = grouped['count'] - grouped['sum']
    grouped['bad'] = grouped['sum']
    grouped['woe'] = np.log((grouped['good'] / grouped['good'].sum()) / (grouped['bad'] / grouped['bad'].sum()))
    grouped['iv'] = (grouped['good'] / grouped['good'].sum() - grouped['bad'] / grouped['bad'].sum()) * grouped['woe']
    return grouped[['woe', 'iv']].sum()

In [22]:
# Calculate WoE & IV for RFMS_Score
woe_iv_values = {}
for col in ['Recency', 'Frequency', 'Total_Transaction_Amount']:
    woe_iv_values[col] = calculate_woe_iv(rfms_df, col, 'RFMS_Score')

C:\Users\smrc\AppData\Local\Temp\ipykernel_24112\3000792547.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(bins)[target].agg(['count', 'sum'])
C:\Users\smrc\AppData\Local\Temp\ipykernel_24112\3000792547.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(bins)[target].agg(['count', 'sum'])
c:\Users\smrc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\smrc\AppData\Local\Programs\Python\Python312\Lib\site-packages\n

In [23]:
woe_iv_values

{'Recency': woe    0.619009
 iv     0.911976
 dtype: float64,
 'Frequency': woe    NaN
 iv     inf
 dtype: float64,
 'Total_Transaction_Amount': woe    inf
 iv     inf
 dtype: float64}